In [1]:
from IPython.display import clear_output, Markdown, display

In [2]:
# No need to run this on colab. These libraries come pre-installed on colab
# %pip install torch torchvision torchaudio nlkt tqdm transformers

# Content:

In this demo, you will Build a very small but functional RAG based QA system

A Rag based QA system is used to answer questions using a defined knowledge base. This knowledge base doesn't have to be a part of the model's training data(meaning we expect that the QA model has no previous knowledge about our knowledge base. This can be the case when we're building a system which answers questions from a companies policies, which are usually kept private from outsiders)

you will Llama-2 to build the system. We can assume that the knowledge base we will be answering questions from is not something llama has existing knowledge of.

Here is how a rag system works in summary:

1. documents are divided into overlapping chunks, converted to embeddings and stored in a vector database (we will not use a vector database and store it in memory)
2. when a user asks a question, that question is converted to embedding.
3. we take the cosine similarity (or another similarity metric) of the question embedding with the document embeddings and pick the top n most similar document chunks (n being a hyper param).
4. we feed the document chunk text along with the user's question to a language model and ask it to answer this question using the info provided.
5. The model scans through the information provided and answers the question. If the model can't find the answer in the most similar one, we assume that the answer to that question doesn't exist in the knowledge base.

## Setup Llama-2

We will not go into details of setting up Llama-2 in this demo



In [3]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_K_M.gguf

clear_output()

In [4]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python==0.2.74  # This takes a few mins when building wheel. Be patient.

clear_output()

In [81]:
import json
import nltk
from nltk.tokenize import regexp_tokenize
from tqdm import tqdm

from llama_cpp import Llama

from transformers import BertModel, BertTokenizer, AutoModel, AutoTokenizer
import torch

from torch.nn.functional import cosine_similarity


nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
model = Llama(
    "llama-2-7b-chat.Q5_K_M.gguf",
    n_gpu_layers=-1,
    n_ctx=2048,
    chat_format="llama-2",
)

clear_output()

## Setting up the knowledge base:

Write a fictional set of rules and regulations. Make sure it has a considerable length. (ideally try to go for 10 to 15 WELL defined rules)

You can write it yourself or you can use another gen AI like chatGPT to write it for you.

The rules can be about something based on an actual fictional world(for example from a book or a movie) or you can invent something entirely new.

If the rules are from an actual fictional world, make sure you modify the rules so our language model can't cheat from the existing knowledge of that fictional setting


One you create the rules, divide your knowledge base into overlapping chunks and convert them into embeddings(you can download and use a pre-trained embedding model for this. You can find state of the art models at hugging face for example.)

## Question Answering

Implement a RAG based QA system. You don't have to actually setup a vector database, since our knowledge base isn't that big, we can keep our embeddings and their respective texts in RAM.

Implement a top n chunks picking system based on similarity to the question(an example of the similarity is cosine similarity) where n can be a hyper parameter. Write prompts for the language model and setup the answering system.

Once the system is setup, ask it a few questions from your knowledge base. Make sure it's a mix of information which exists in the knowledge base and information which doesn't.

Show how your model performs against your questions